# [CS303]Practice 9 - Min-Conflict
In Practice 6 and 7, we have applied the BTS algorithm and the Improving BTS to solve the N-Queens problem.

In this practice, you are required to implement the `min_conflict()` function using the **Min-Conflict algorithm**. Your solution should run efficiently — it must complete **within 2s** when **N = 100 (no GUI)**.

📌 **DDL: Nov.30th**

## Evaluation Criteria

Your submission will be assessed on:

* Clear understanding of every line of your code (no blind copy-paste).
* Successful compilation of the program.
* Correctness of the program logic.
* Reasonable efficiency of the solution.

### Grading Policy

* **On-time submission:** 60–100 points
* **Late submission:** 0 points


In [1]:
import numpy as np
import time

def my_range(start, end):
    if start <= end:
        return range(start, end + 1)
    else:
        return range(start, end - 1, -1)


class Problem:
    char_mapping = ('·', 'Q')

    def __init__(self, n=4):
        self.N = n

    def is_valid(self, state):
        """
        check whether the state satisfies the condition or not.
        : param state: list of points (in (row id, col id) tuple form)
        : return: bool value of valid or not
        """
        board = self.get_board(state)
        res = True
        for point in state:
            i, j = point
            condition1 = board[:, j].sum() <= 1
            condition2 = board[i, :].sum() <= 1
            condition3 = self.pos_slant_condition(board, point)
            condition4 = self.neg_slant_condition(board, point)
            res = res and condition1 and condition2 and condition3 and condition4
            if not res:
                break
        return res

    def is_satisfy(self, state):
        return self.is_valid(state) and len(state) == self.N

    def pos_slant_condition(self, board, point):
        i, j = point
        tmp = min(self.N - i - 1, j)
        start = (i + tmp, j - tmp)
        tmp = min(i, self.N - j - 1)
        end = (i - tmp,  j + tmp)
        rows = my_range(start[0], end[0])
        cols = my_range(start[1], end[1])
        return board[rows, cols].sum() <= 1

    def neg_slant_condition(self, board, point):
        i, j = point
        tmp = min(i, j)
        start = (i - tmp, j - tmp)
        tmp = min(self.N - i - 1, self.N - j - 1)
        end = (i + tmp, j + tmp)
        rows = my_range(start[0], end[0])
        cols = my_range(start[1], end[1])
        return board[rows, cols].sum() <= 1

    def get_board(self, state):
        board = np.zeros([self.N, self.N], dtype=int)
        for point in state:
            board[point] = 1
        return board

    def print_state(self, state):
        board = self.get_board(state)
        print('_' * (2 * self.N + 1))
        for row in board:
            for item in row:
                print(f'|{Problem.char_mapping[item]}', end='')
            print('|')
        print('-' * (2 * self.N + 1))

In [2]:
class Solver:
    def __init__(self, n, method):
        self.N = n
        self.problem = Problem(n)
        self.method = method

    def solve(self, render=True):
        if render:
            import pygame
            w, h = 90 * self.n + 10, 90 * self.N + 10
            screen = pygame.display.set_mode((w, h))
            screen.fill('white')
            action_generator = self.method(self.problem)
            clk = pygame.time.Clock()
            queen_img = pygame.image.load('./queen.png')
            while True:
                for event in pygame.event.get():
                    if event == pygame.QUIT:
                        exit()
                try:
                    actions = next(action_generator)
                    screen.fill('white')
                    for i in range(self.n + 1):
                        pygame.draw.rect(screen, 'black', (i * 90, 0, 10, h))
                        pygame.draw.rect(screen, 'black', (0, i * 90, w, 10))
                    for action in actions:
                        i, j = action
                        screen.blit(queen_img, (10 + 90 * j, 10 + 90 * i))
                    pygame.display.flip()
                except StopIteration:
                    pass
                clk.tick(5)
            pass
        else:
            start_time = time.time()
            for actions in self.method(self.problem):
                pass
            self.problem.print_state(actions)
            print(time.time() - start_time)

## Min-Conflict algorithm (Local search for CSP)

* First generate a complete assignment for all variables (this set of assignments may conflict)

* Repeat the following steps until there are no conflicts:

    * Randomly Select a variable that causes conflicts

    * Reassign the value of this variable to another value that with the least constraint onflicts with other variables

In [3]:
def check_conflict(problem, state):
    row_conflicts = np.zeros(problem.N, dtype=int)
    for row in range(problem.N):
        col = next(c for r, c in state if r == row)
        for r, c in state:
            if r != row and (c == col or abs(r-row) == abs(c-col)):
                row_conflicts[row] += 1
    return row_conflicts

def min_conflict(problem):
    # init
    action_stack = [(row, np.random.randint(0, problem.N)) for row in range(problem.N)]
    row_conflicts = check_conflict(problem, action_stack)
    yield action_stack

    conflict_rows = {row for row, conflict in enumerate(row_conflicts) if conflict > 0}

    while conflict_rows:
        # random conflict row, better than max conflict row because time complexity
        current_row = np.random.choice(list(conflict_rows))

        # min conflict column
        min_conflict = problem.N + 1
        best_cols = []
        for new_col in range(problem.N):
            conflict_count = sum(
                1 for r, c in action_stack
                if r != current_row and (c == new_col or abs(r - current_row) == abs(c - new_col))
            )
            if conflict_count < min_conflict:
                min_conflict = conflict_count
                best_cols = [new_col]
            elif conflict_count == min_conflict:
                best_cols.append(new_col)

        # update state
        chosen_col = np.random.choice(best_cols)
        action_stack[current_row] = (current_row, chosen_col)
        row_conflicts = check_conflict(problem, action_stack)
        conflict_rows = {row for row, conflict in enumerate(row_conflicts) if conflict > 0}

        yield action_stack

## test block

In [4]:
Solver(n=100, method=min_conflict).solve(render=False)

_________________________________________________________________________________________________________________________________________________________________________________________________________
|·|·|·|·|·|·|·|·|Q|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|
|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|Q|·|·|·|·|
|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|Q|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|
|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|Q|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·